# Prepare Gibbs-Duhem Simulations

In [1]:
import os, sys 
import numpy as np

In [2]:
settings=dict()

settings['--step']=250
settings['--initial_equilibration_steps']=200000
settings['--steps_per_sim']=100000

settings['--percent_equilibration']=20.0
settings['--max_error_vol']='0.05'
settings['--lmp_exe']='$LAMMPS_EXE'
settings['--run_cmd']='"srun"'
settings['--lmp_options']='""'
settings['--out_freq']=10000
settings['--root_fold']='./'
settings['--max_num_runs']='250'
settings['--units']='metal'

settings_T=settings.copy()
settings_T['--integration_variable']= 'T'
settings_T['--step']=5


In [3]:
def PrepSingleSim(setttings_temp,dir_name,left_phase,right_phase,pm,tm,pmin,pmax):
    dir_name=dir_name+'/'
    # Create simulation dir
    root_folder=dir_name+'/'+str(pm)+'atm'+'_'+str(tm)+'K'
    os.makedirs(dir_name, exist_ok=True)#s.system('mkdir  {}'.format(dir_name))
    os.makedirs(dir_name+'/Initial', exist_ok=True)
    os.makedirs(root_folder, exist_ok=True)

    os.system('cp integrate.py ' + dir_name)
    os.system('cp ../../2-GenerateTrainingset/GraphFiles/Iteration1/graph-compress.pb ' + dir_name+'/graph.pb')
    initial='Initial/'
    os.system('cp -r InitialConfigurations/{} {}{}'.format(left_phase,dir_name, initial) )
    os.system('cp -r InitialConfigurations/{} {}{}'.format(right_phase,dir_name, initial))

    
    
    # Setup command for running

    setttings_temp['--end_variable']=str(pmax)
    setttings_temp['--initial_point_folder']='../'+initial
    setttings_temp['--left']=left_phase
    setttings_temp['--right']=right_phase
    setttings_temp['--initial_TP']=str(tm)+' '+str(pm) 
    arguments=' '.join(['{} {}'.format(t,setttings_temp[t]) for t in setttings_temp])
    
    # Upwards
 
    arguments=' '.join(['{} {}'.format(t,setttings_temp[t]) for t in setttings_temp])
    command = 'python3 ../integrate.py {}\n'.format(arguments) 
    
    #Downwards
    setttings_temp['--end_variable']=str(pmin)
    arguments=' '.join(['{} {}'.format(t,setttings_temp[t]) for t in setttings_temp])
    command += 'python3 ../integrate.py {}\n'.format(arguments) 
    
    with open(root_folder+'/script.sh','w') as fp:
        fp.write(command)
    

## DNN@MB-pol

### IceIh-Liquid

In [13]:
for PT in np.loadtxt('../IceIh-Liquid/3-BiasedCoexistence/DATA/melting-points-DNN.dat'):
    print(PT)
    PrepSingleSim(settings_T,'Ih-Liq_NN/','IceIh','Liquid',PT[0],PT[1],100,PT[1])
    PrepSingleSim(settings,'Ih-Liq_NN/','IceIh','Liquid',PT[0],PT[1],-1000,PT[0])


[  1.   262.49]
[1000.    250.35]


### IceII-Liquid

In [14]:
for PT in np.atleast_2d(np.loadtxt('../IceII-Liquid/5-BiasedCoexistence_3rd/DATA/melting-points-DNN.dat')):
    print(PT)
    PrepSingleSim(settings,'II-Liq_NN/','IceII','Liquid',PT[0],PT[1],0,8000)

[2000.    244.62]
[3000.    250.88]
[4000.    254.27]
[5000.    255.18]
[6000.    256.12]


### IceIII-Liquid

In [16]:
for PT in np.loadtxt('../IceIII-Liquid/5-BiasedCoexistence/DATA/melting-points-DNN.dat'):
    print(PT)
    PrepSingleSim(settings,'III-Liq_NN/','IceIII','Liquid',PT[0],PT[1],0,10000)

[2000.    243.15]
[3000.    245.15]
[5000.    247.73]
[6000.    247.37]


### IceV-Liquid

In [18]:
for PT in np.loadtxt('../IceV-Liquid/3-BiasedCoexistence/DATA/melting-points-DNN.dat'):
    print(PT)
    PrepSingleSim(settings,'V-Liq_NN/','IceV','Liquid',PT[0],PT[1],2000,12000)

[5000.    249.96]
[6000.   255.2]


### IceVI-Liquid

In [20]:
for PT in np.loadtxt('../IceVI-Liquid/4-BiasedCoexistence/DATA/melting-points-DNN.dat'):
    print(PT)
    PrepSingleSim(settings,'VI-Liq_NN/','IceVI','Liquid',PT[0],PT[1],3000,30000)

[7000.    263.12]
[9000.    276.99]


### IceVI-IceV

In [15]:
with open('../FullPhaseDiagram/triple_points_NN.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'Liquid IceV IceVI' in line:
            PrepSingleSim(settings_T,'VI-V_NN/','IceVI','IceV',float(PT[-1]),float(PT[-2]),100,PT[-2])

### IceII-IceVI

In [16]:
with open('../FullPhaseDiagram/triple_points_NN.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceII IceVI' in line:
            PrepSingleSim(settings_T,'II-VI_NN/','IceII','IceVI',float(PT[-1]),float(PT[-2]),50,PT[-2])

### IceII IceIh

In [17]:
with open('../FullPhaseDiagram/triple_points_NN.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceIh IceII' in line:
            PrepSingleSim(settings_T,'II-Ih_NN/','IceII','IceIh',float(PT[-1]),float(PT[-2]),0,float(PT[-2]))

## IceIII IceV

In [21]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceIII IceV' in line:
            PrepSingleSim(settings_T,'V-III/','IceV','IceIII',float(PT[-1]),float(PT[-2]),50,float(PT[-2]))

## IceII IceIh

In [22]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceIh IceII' in line:
            PrepSingleSim(settings_T,'II-Ih/','IceII','IceIh',float(PT[-1]),float(PT[-2]),0,float(PT[-2]))

cp: InitialConfigurations/IceII/.#in.dump: No such file or directory


## IceIII IceIh

In [ ]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceIII IceIh' in line:
            PrepSingleSim(settings_T,'IceIh_IceIII/','IceIh','IceIII',float(PT[-1]),float(PT[-2]),0,250)

## IceII IceIII

In [14]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'IceII IceIII IceV' in line or 'IceII IceIh IceIII' in line:
            PrepSingleSim(settings,'IceII_IceIII_2nd/','IceII','IceIII',float(PT[-1]),float(PT[-2]),12000,-2000)

## IceII IceV

In [8]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'IceII IceIII IceV' in line:
            PrepSingleSim(settings_T,'IceII_IceV/','IceII','IceV',float(PT[-1]),float(PT[-2]),float(PT[-2]),0)

## IceII IceVI

In [9]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'IceII IceV IceVI' in line:
            PrepSingleSim(settings_T,'IceII_IceVI/','IceII','IceVI',float(PT[-1]),float(PT[-2]),float(PT[-2]),0)

## IceII IceVI v2

In [10]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceII IceVI' in line:
            PrepSingleSim(settings_T,'IceII_IceVI_v2/','IceII','IceVI',float(PT[-1]),float(PT[-2]),300,0)

## IceII IceV

In [4]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceII IceV' in line:
            PrepSingleSim(settings_T,'IceII_IceV_2nd/','IceII','IceV',float(PT[-1]),float(PT[-2]),300,0)

## Classical MB-pol

### IceIh-Liquid

In [34]:
for PT in np.loadtxt('../IceIh-Liquid/3-BiasedCoexistence/DATA/melting-points-MB-pol-classical.dat'):
    print(PT)
    PrepSingleSim(settings_T,'Ih-Liq_classical/','IceIh','Liquid',PT[0],PT[1],100,PT[1])
    PrepSingleSim(settings,'Ih-Liq_classical/','IceIh','Liquid',PT[0],PT[1],-1000,PT[0])


[  1.   266.22]
[1000.    255.01]


### IceII-Liquid

In [35]:
for PT in np.atleast_2d(np.loadtxt('../IceII-Liquid/5-BiasedCoexistence_3rd/DATA/melting-points-MB-pol-classical.dat')):
    print(PT)
    PrepSingleSim(settings,'II-Liq_classical/','IceII','Liquid',PT[0],PT[1],0,8000)

[2000.    244.62]
[3000.    251.02]
[4000.    254.43]
[5000.   255.5]
[6000.    256.58]


### IceIII-Liquid

In [36]:
for PT in np.loadtxt('../IceIII-Liquid/5-BiasedCoexistence/DATA/melting-points-MB-pol-classical.dat'):
    print(PT)
    PrepSingleSim(settings,'III-Liq_classical/','IceIII','Liquid',PT[0],PT[1],0,10000)

[2000.    247.39]
[3000.    248.96]
[5000.   250.5]
[6000.    250.21]


### IceV-Liquid

In [37]:
for PT in np.loadtxt('../IceV-Liquid/3-BiasedCoexistence/DATA/melting-points-MB-pol-classical.dat'):
    print(PT)
    PrepSingleSim(settings,'V-Liq_classical/','IceV','Liquid',PT[0],PT[1],2000,12000)

[5000.    255.82]
[6000.    260.26]


### IceVI-Liquid

In [11]:
for PT in np.loadtxt('../IceVI-Liquid/4-BiasedCoexistence/DATA/melting-points-MB-pol-classical.dat'):
    print(PT)
    PrepSingleSim(settings,'VI-Liq_classical/','IceVI','Liquid',PT[0],PT[1],3000,15000)

[7000.    266.15]
[9000.   279.9]


### IceVI-IceV

In [18]:
with open('../FullPhaseDiagram/triple_points_classical.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'Liquid IceV IceVI' in line:
            PrepSingleSim(settings_T,'VI-V_classical/','IceVI','IceV',float(PT[-1]),float(PT[-2]),100,PT[-2])

### IceII-IceV

In [19]:
with open('../FullPhaseDiagram/triple_points_classical.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'Liquid IceII IceV' in line:
            PrepSingleSim(settings_T,'II-V_classical/','IceII','IceV',float(PT[-1]),float(PT[-2]),100,PT[-2])

### IceII-IceIII

In [20]:
with open('../FullPhaseDiagram/triple_points_classical.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'Liquid IceII IceIII' in line:
            PrepSingleSim(settings_T,'II-III_classical/','IceII','IceIII',float(PT[-1]),float(PT[-2]),100,PT[-2])

### IceIh-IceIII

In [21]:
with open('../FullPhaseDiagram/triple_points_classical.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'Liquid IceIh IceIII' in line:
            PrepSingleSim(settings_T,'Ih-III_classical/','IceIh','IceIII',float(PT[-1]),float(PT[-2]),100,PT[-2])

### Ih-II

In [26]:
with open('../FullPhaseDiagram/triple_points_classical.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'IceIh IceII IceIII' in line:
            PrepSingleSim(settings_T,'Ih-II_classical/','IceIh','IceII',float(PT[-1]),float(PT[-2]),100,PT[-2])

### II-VI

In [27]:
with open('../FullPhaseDiagram/triple_points_classical.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'IceII IceV IceVI' in line:
            PrepSingleSim(settings_T,'II-VI_classical/','IceII','IceVI',float(PT[-1]),float(PT[-2]),100,PT[-2])

## Quantum

In [39]:
for PT in np.loadtxt('../IceIh-Liquid/3-BiasedCoexistence/DATA/melting-points-MB-pol-quantum.dat'):
    print(PT)
    PrepSingleSim(settings_T,'Ih-Liq_quantum/','IceIh','Liquid',PT[0],PT[1],100,PT[1])
    PrepSingleSim(settings,'Ih-Liq_quantum/','IceIh','Liquid',PT[0],PT[1],-1000,PT[0])


[  1.   262.35]
[1000.    250.37]


### IceII-Liquid

In [40]:
for PT in np.atleast_2d(np.loadtxt('../IceII-Liquid/5-BiasedCoexistence_3rd/DATA/melting-points-MB-pol-quantum.dat')):
    print(PT)
    PrepSingleSim(settings,'II-Liq_quantum/','IceII','Liquid',PT[0],PT[1],0,8000)

[2000.    243.22]
[3000.    249.47]
[4000.    252.08]
[5000.    252.34]
[6000.    254.93]


### IceIII-Liquid

In [41]:
for PT in np.loadtxt('../IceIII-Liquid/5-BiasedCoexistence/DATA/melting-points-MB-pol-quantum.dat'):
    print(PT)
    PrepSingleSim(settings,'III-Liq_quantum/','IceIII','Liquid',PT[0],PT[1],0,10000)

[2000.   247.5]
[3000.    248.35]
[5000.    250.84]
[6000.    249.93]


### IceV-Liquid

In [42]:
for PT in np.loadtxt('../IceV-Liquid/3-BiasedCoexistence/DATA/melting-points-MB-pol-quantum.dat'):
    print(PT)
    PrepSingleSim(settings,'V-Liq_quantum/','IceV','Liquid',PT[0],PT[1],2000,12000)

[5000.    257.54]
[6000.    262.23]


### IceVI-Liquid

In [43]:
for PT in np.loadtxt('../IceVI-Liquid/4-BiasedCoexistence/DATA/melting-points-MB-pol-quantum.dat'):
    print(PT)
    PrepSingleSim(settings,'VI-Liq_quantum/','IceVI','Liquid',PT[0],PT[1],3000,15000)

[7000.    268.38]
[9000.    283.63]


### IceVI-IceV

In [22]:
with open('../FullPhaseDiagram/triple_points_quantum.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'Liquid IceV IceVI' in line:
            PrepSingleSim(settings_T,'VI-V_quantum/','IceVI','IceV',float(PT[-1]),float(PT[-2]),100,PT[-2])

### IceII-IceV

In [23]:
with open('../FullPhaseDiagram/triple_points_quantum.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'Liquid IceII IceV' in line:
            PrepSingleSim(settings_T,'II-V_quantum/','IceII','IceV',float(PT[-1]),float(PT[-2]),100,PT[-2])

### IceII-IceIII

In [24]:
with open('../FullPhaseDiagram/triple_points_quantum.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'Liquid IceII IceIII' in line:
            PrepSingleSim(settings_T,'II-III_quantum/','IceII','IceIII',float(PT[-1]),float(PT[-2]),100,PT[-2])

### IceIh-IceIII

In [25]:
with open('../FullPhaseDiagram/triple_points_quantum.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'Liquid IceIh IceIII' in line:
            PrepSingleSim(settings_T,'Ih-III_quantum/','IceIh','IceIII',float(PT[-1]),float(PT[-2]),100,PT[-2])

### Ih-II

In [28]:
with open('../FullPhaseDiagram/triple_points_quantum.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'IceIh IceII IceIII' in line:
            PrepSingleSim(settings_T,'Ih-II_quantum/','IceIh','IceII',float(PT[-1]),float(PT[-2]),100,PT[-2])

### II-VI

In [30]:
with open('../FullPhaseDiagram/triple_points_quantum.dat','r') as fp:
    lines=fp.readlines()
    for line in lines:
        PT=line.split()
        if 'IceII IceV IceVI' in line:
            PrepSingleSim(settings_T,'II-VI_quantum/','IceII','IceVI',float(PT[-1]),float(PT[-2]),100,PT[-2])

In [ ]:
!jupyter nbconvert --to script PrepareGibbsDuhem.ipynb